## Deliverable 2. Create a Customer Travel Destinations Map.

In [36]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps


# Import API key
from config import g_key

In [17]:
# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [18]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Cape Town,ZA,-33.9258,18.4232,70.92,45,0,9.22,clear sky
1,1,Chardara,KZ,41.2547,67.9692,77.94,48,100,6.13,overcast clouds
2,2,Amuntai,ID,-2.4177,115.2494,73.99,98,48,1.01,scattered clouds
3,3,Atar,MR,20.5169,-13.0499,110.66,9,22,8.12,few clouds
4,4,Airai,TL,-8.9266,125.4092,56.12,86,83,3.40,broken clouds


In [21]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [24]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
clean_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
clean_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Chardara,KZ,41.2547,67.9692,77.94,48,100,6.13,overcast clouds
10,10,Lata,PT,40.1629,-8.3327,85.50,50,39,9.19,scattered clouds
16,16,Puerto Ayora,EC,-0.7393,-90.3518,75.16,92,74,5.01,broken clouds
18,18,Boyolangu,ID,-8.1181,111.8935,76.37,91,77,2.39,broken clouds
25,25,San Andres,CO,12.5847,-81.7006,82.42,83,75,11.50,broken clouds
26,26,Victoria,HK,22.2855,114.1577,83.32,89,100,10.00,overcast clouds
33,33,Eyl,SO,7.9803,49.8164,79.83,77,100,26.46,overcast clouds
35,35,Atuona,PF,-9.8000,-139.0333,77.20,69,36,15.35,scattered clouds
37,37,Tianmen,CN,30.6516,113.1603,76.30,71,100,1.81,light rain
38,38,Cabo San Lucas,MX,22.8909,-109.9124,86.61,75,26,6.20,scattered clouds


In [25]:
# 4a. Determine if there are any empty rows.
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [26]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.

In [27]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Chardara,KZ,77.94,overcast clouds,41.2547,67.9692,
10,Lata,PT,85.50,scattered clouds,40.1629,-8.3327,
16,Puerto Ayora,EC,75.16,broken clouds,-0.7393,-90.3518,
18,Boyolangu,ID,76.37,broken clouds,-8.1181,111.8935,
25,San Andres,CO,82.42,broken clouds,12.5847,-81.7006,
26,Victoria,HK,83.32,overcast clouds,22.2855,114.1577,
33,Eyl,SO,79.83,overcast clouds,7.9803,49.8164,
35,Atuona,PF,77.20,scattered clouds,-9.8000,-139.0333,
37,Tianmen,CN,76.30,light rain,30.6516,113.1603,
38,Cabo San Lucas,MX,86.61,scattered clouds,22.8909,-109.9124,


In [28]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows(): 

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()

    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [29]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Chardara,KZ,77.94,overcast clouds,41.2547,67.9692,Жанаргул
10,Lata,PT,85.50,scattered clouds,40.1629,-8.3327,Casa Maquia
16,Puerto Ayora,EC,75.16,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
18,Boyolangu,ID,76.37,broken clouds,-8.1181,111.8935,Yos ega cv
25,San Andres,CO,82.42,broken clouds,12.5847,-81.7006,Hotel Arena Blanca
26,Victoria,HK,83.32,overcast clouds,22.2855,114.1577,Mini Central
33,Eyl,SO,79.83,overcast clouds,7.9803,49.8164,Amiin Hotel
35,Atuona,PF,77.20,scattered clouds,-9.8000,-139.0333,Villa Enata
37,Tianmen,CN,76.30,light rain,30.6516,113.1603,Tianmen Luyu Hotel
38,Cabo San Lucas,MX,86.61,scattered clouds,22.8909,-109.9124,Vista Roof Restaurant


In [31]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Chardara,KZ,77.94,overcast clouds,41.2547,67.9692,Жанаргул
10,Lata,PT,85.50,scattered clouds,40.1629,-8.3327,Casa Maquia
16,Puerto Ayora,EC,75.16,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
18,Boyolangu,ID,76.37,broken clouds,-8.1181,111.8935,Yos ega cv
25,San Andres,CO,82.42,broken clouds,12.5847,-81.7006,Hotel Arena Blanca


In [33]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [34]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [35]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))